# Assignment: Data Wrangling
## `! git clone https://github.com/DS3001/wrangling`
## Do Q2, and one of Q1 or Q3.

**Q1.** Open the "tidy_data.pdf" document in the repo, which is a paper called Tidy Data by Hadley Wickham.

  1. Read the abstract. What is this paper about?
  2. Read the introduction. What is the "tidy data standard" intended to accomplish?
  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."
  4. Read Section 2.2. How does Wickham define values, variables, and observations?
  5. How is "Tidy Data" defined in section 2.3?
  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?
  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?
  8. Read Section 6. What is the "chicken-and-egg" problem with focusing on tidy data? What does Wickham hope happens in the future with further work on the subject of data wrangling?

**Q2.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)

When I tried coercing the Price values without replacing commas with the empty string, any value that exceeded 3 digits were truncated because the comma that was in the number greater than 999 caused the number to be treated as a string. Thus, I had to get rid of the commas for the numbers that exceeded 999 for them to be treated properly during the typecast step. I ended up with 0 missing values after cleaning the variable.



2. Categorical variable: For the `./data/sharks.csv` data covered in the lecture, clean the "Type" variable as well as you can, and explain the choices you make.

I wanted to combine some of the Type categories to avoid redundancy. For example, several types were related to watercrafts (e.g. "Boating" and "Boat"), and they could easily be merged into one general category. A similar phenomenon occurred with the types related to unknown/incomplete values, so it was more practical to merge those types to just be "nan" values.

3. Dummy variable: For the pretrial data covered in the lecture, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

I looked in the pretrial data codebook, and the SentenceTypeAllChargesAtConvictionInContactEvent variable has a code of 4 if "charges in cohort contact resulted in pending, dismissed, deferred, etc." which would indicate that the ImposedSentenceAllChargeInContactEvent (i.e. the total imposed sentence term for any charges) should be 0. Thus, for the "ImposedSentenceAllChargeInContactEvent" dataframe, I replaced the values with 0 when the "SentenceTypeAllChargesAtConvictionInContactEvent" value is type 4.

In [14]:
# ------ code for q2.1 ------
import numpy as np # Import the numpy package into your workspace
import matplotlib.pyplot as plt # Import matplotlib into your workspace
import pandas as pd  # Import the pandas package into your workspace

df = pd.read_csv('/home/airbnb_hw.csv',low_memory=False)
df.head()

price_df = df["Price"]

print('Before coercion: \n', price_df .describe(),'\n') # price variable not read by pandas as a number
# df[var].hist(bins=50) # Initial histogram; check out the x axis

# before coercion, need to get rid of comma separators like in 8,000 --> coercing without this step
# causes all values to be numeric BUT only 3 digits long

price_df = price_df.str.replace(",", "") # replace commas w/ empty string
print(price_df.unique())

# coerce Price to numeric
price_df = pd.to_numeric(price_df , errors='coerce') # Coerce the variable to numeric

# after coercion
print('After coercion: \n', price_df.describe(),'\n') # price variable not read by pandas as a number

df.head()

print(price_df.unique())

print( 'Total missing: ', sum( price_df.isnull() ) ) # This converts all the values


Before coercion: 
 count     30478
unique      511
top         150
freq       1481
Name: Price, dtype: object 

['145' '37' '28' '199' '549' '149' '250' '90' '270' '290' '170' '59' '49'
 '68' '285' '75' '100' '150' '700' '125' '175' '40' '89' '95' '99' '499'
 '120' '79' '110' '180' '143' '230' '350' '135' '85' '60' '70' '55' '44'
 '200' '165' '115' '74' '84' '129' '50' '185' '80' '190' '140' '45' '65'
 '225' '600' '109' '1990' '73' '240' '72' '105' '155' '160' '42' '132'
 '117' '295' '280' '159' '107' '69' '239' '220' '399' '130' '375' '585'
 '275' '139' '260' '35' '133' '300' '289' '179' '98' '195' '29' '27' '39'
 '249' '192' '142' '169' '1000' '131' '138' '113' '122' '329' '101' '475'
 '238' '272' '308' '126' '235' '315' '248' '128' '56' '207' '450' '215'
 '210' '385' '445' '136' '247' '118' '77' '76' '92' '198' '205' '299'
 '222' '245' '104' '153' '349' '114' '320' '292' '226' '420' '500' '325'
 '307' '78' '265' '108' '123' '189' '32' '58' '86' '219' '800' '335' '63'
 '229' '425' '6

In [19]:
# ------ code for q2.2 ------
import numpy as np # Import the numpy package into your workspace
import matplotlib.pyplot as plt # Import matplotlib into your workspace
import pandas as pd  # Import the pandas package into your workspace

df = pd.read_csv('/home/sharks.csv',low_memory=False)
df.head()

sharks_df = df["Type"]
sharks_df.value_counts() # tabulate num of times each unique val occurs

# combine all values related to boating
sharks_df = sharks_df.replace(["Sea Disaster", "Boat", "Boating", "Boatomg"], "Watercraft")
sharks_df.value_counts()

# combine all vals related to unknown
sharks_df = sharks_df.replace(["Invalid", "Questionable", "Unconfirmed", "Unverified", "Under investigation"], np.nan)
sharks_df.value_counts()




,count
Type,
Unprovoked,4716
Provoked,593
Watercraft,583


In [25]:
# ------ code for q2.3 ------
import numpy as np # Import the numpy package into your workspace
import matplotlib.pyplot as plt # Import matplotlib into your workspace
import pandas as pd  # Import the pandas package into your workspace

df = pd.read_csv('/home/pretrial_data.csv',low_memory=False)
df.head()

df.columns.tolist()
released_df = df["released"]
released_df.unique()

# replace missing vals w/ np.nan
released_df = released_df.replace(9, np.nan) # 9's are unclear
released_df.value_counts()


,count
released,
1.0,19154
0.0,3801


In [34]:
# ------ code for q2.4 ------
import numpy as np # Import the numpy package into your workspace
import matplotlib.pyplot as plt # Import matplotlib into your workspace
import pandas as pd  # Import the pandas package into your workspace

df = pd.read_csv('/home/pretrial_data.csv',low_memory=False)
df.head()

imposed_df = df["ImposedSentenceAllChargeInContactEvent"]
sen_type_df = df["SentenceTypeAllChargesAtConvictionInContactEvent"]
print(imposed_df.unique())

imposed_df = pd.to_numeric(imposed_df, errors="coerce") # cast to numeric
print(imposed_df.unique())

# create missing dummy
imposed_na = imposed_df.isnull()
print(np.sum(imposed_na),'\n')

print(pd.crosstab(imposed_na, sen_type_df), '\n') # category 4 = charges dismissed

# replace vals for imposed_df w/ False (0) when sen_type = 4
imposed_df = imposed_df.mask(sen_type_df == 4, 0)

# replace vals for imposed_df w/ np.nan when sen_type = 9
imposed_df = imposed_df.mask(sen_type_df == 9, np.nan)

# create missing dummy
imposed_na = imposed_df.isnull()
print(pd.crosstab(imposed_na, sen_type_df), "\n")
print(np.sum(imposed_na), "\n")



[' ' '60' '12' '.985626283367556' '36' '6' '24' '5.91375770020534' '120'
 '72' '11.9917864476386' '0' '2.95687885010267' '84' '108' '300' '240'
 '180' '4' '96' '2' '54' '.328542094455852' '44' '5' '115' '132' '48'
 '258' '34' '76' '.164271047227926' '.131416837782341' '111' '9' '3'
 '1.97125256673511' '36.9856262833676' '.0657084188911704'
 '35.4928131416838' '106.492813141684' '8' '35' '18.3141683778234' '480'
 '32' '93' '234' '732' '1.16427104722793' '4.6570841889117' '21' '7'
 '4.49281314168378' '18' '600' '43.1642710472279' '179' '52' '30' '20'
 '192' '702' '14' '55' '53' '11.9055441478439' '114' '35.0061601642711'
 '68' '.657084188911704' '46.6242299794661' '102' '65' '200' '57'
 '24.3285420944559' '12.1642710472279' '117' '81.4928131416838'
 '22.4928131416838' '1980' '3.6570841889117' '56' '10' '2.79260780287474'
 '1' '47' '22' '1500' '40' '284' '11' '118' '42' '162' '156'
 '47.2956878850103' '105' '51' '246' '29' '75' '324' '360'
 '34.4804928131417' '120.328542094456' '59.926078

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Q3.** Many important datasets contain a race variable, typically limited to a handful of values often including Black, White, Asian, Latino, and Indigenous. This question looks at data gathering efforts on this variable by the U.S. Federal government.

1. How did the most recent US Census gather data on race?
2. Why do we gather these data? What role do these kinds of data play in politics and society? Why does data quality matter?
3. Please provide a constructive criticism of how the Census was conducted: What was done well? What do you think was missing? How should future large scale surveys be adjusted to best reflect the diversity of the population? Could some of the Census' good practices be adopted more widely to gather richer and more useful data?
4. How did the Census gather data on sex and gender? Please provide a similar constructive criticism of their practices.
5. When it comes to cleaning data, what concerns do you have about protected characteristics like sex, gender, sexual identity, or race? What challenges can you imagine arising when there are missing values? What good or bad practices might people adopt, and why?
6. Suppose someone invented an algorithm to impute values for protected characteristics like race, gender, sex, or sexuality. What kinds of concerns would you have?